In [18]:
!pip install gluonts
!pip install mxnet-mkl==1.6.0 numpy==1.23.1
!pip install "gluonts[torch]"

In [4]:
import pandas as pd
import numpy as np

def convert_to_arrow(csv_path: str):
    """
    Store a given set of series into Arrow format at the specified path.

    Input data can be a CSV file with a 'close' column.
    """
    # Baca file CSV
    data = pd.read_csv(csv_path, parse_dates=['timestamp'])

    # Hitung jumlah total baris
    total_rows = len(data)

    # Hitung 70% dari total baris
    rows_to_take = int(0.7 * total_rows)

    # Ambil 70% pertama dari data
    data = data.head(rows_to_take)

    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.sort_values(by='timestamp')
    
    # Ambil array dari kolom 'close'
    time_series = data['close'].to_numpy()
    
    # Set an arbitrary start time
    start = np.datetime64(data['timestamp'].iloc[0], "s")

    # Buat dataset yang terdiri dari satu seri waktu dengan satu start time
    dataset = {"start": start, "target": time_series}

    return dataset

# 42 LQ45
dataset_name=["ACES","AMRT","ASII","BBRI","BRIS","CPIN","GGRM","ICBP",
              "INKP","ITMG","MDKA","PGAS","SMGR","TOWR","ADRO","ANTM",
              "BBCA","BBTN","BRPT","ESSA","GOTO","INCO","INTP","KLBF",
              "MEDC","PTBA","SRTG","UNTR","AKRA","ARTO","BBNI","BMRI",
              "BUKA","EXCL","HRUM","INDF","ISAT","MAPI","MTEL","SIDO",
              "TLKM","UNVR"]

dataset_train_model =[]

count=0
for ds in dataset_name:
  # Sesuaikan path ke lokasi file CSV dan output yang diinginkan


  data = pd.read_csv(f"/home/yogi/chronos-research/dataset/LQ45-daily/{ds}.csv", parse_dates=['timestamp'])

  # Hitung jumlah total baris
  total_rows = len(data)

  print(ds,"= ",total_rows)
  count+=1

  dataset_train_model.append(convert_to_arrow(f"/home/yogi/chronos-research/dataset/LQ45-daily/{ds}.csv"))
count

ACES =  3959
AMRT =  3647
ASII =  5670
BBRI =  5000
BRIS =  1218
CPIN =  5670
GGRM =  5670
ICBP =  3197
INKP =  5670
ITMG =  3929
MDKA =  1971
PGAS =  4975
SMGR =  5670
TOWR =  3350
ADRO =  3778
ANTM =  5670
BBCA =  5670
BBTN =  3407
BRPT =  5670
ESSA =  2853
GOTO =  195
INCO =  5670
INTP =  5670
KLBF =  5670
MEDC =  5670
PTBA =  5230
SRTG =  2488
UNTR =  5670
AKRA =  5670
ARTO =  1824
BBNI =  5670
BMRI =  5085
BUKA =  371
EXCL =  4507
HRUM =  3198
INDF =  5670
ISAT =  5670
MAPI =  4738
MTEL =  295
SIDO =  2363
TLKM =  5670
UNVR =  5670


42

In [ ]:
import pandas as pd
import numpy as np

from gluonts.dataset.split import split
from gluonts.evaluation import Evaluator, make_evaluation_predictions
from gluonts.torch import DeepAREstimator
from gluonts.dataset.common import ListDataset


def load_and_process_data(
    filename,
    date_column_name,
    index_timezone="America/New_York",
    fillna_method="ffill",
):
    # Load the data
    df = pd.read_csv(filename)

    # Ensure the Date column is a datetime object
    df[date_column_name] = pd.to_datetime(df[date_column_name], utc=True)

    # Convert the timezone of the Date column to the specified timezone
    df[date_column_name] = df[date_column_name].dt.tz_convert(index_timezone)

    # Set Date column as the index
    df.set_index(date_column_name, inplace=True)

    # If Adj Close column exists, keep only this column and drop others
    if "Adj Close" in df.columns:
        df = df[["Adj Close"]]
        # Rename 'Adj Close' to be more specific based on the filename
        new_column_name = filename.split("/")[-1].split("_")[0] + "_Adj_Close"
        df.rename(columns={"Adj Close": new_column_name}, inplace=True)

    # If fillna method is specified, fill the missing values
    if fillna_method:
        df.fillna(method=fillna_method, inplace=True)

    # Check if there are still NaN values and backfill if needed
    if df.isnull().sum().any():
        df.fillna(method="bfill", inplace=True)

    return df


df = load_and_process_data(
    "/home/yogi/chronos-research/dataset/LQ45-daily/ANTM.csv", "timestamp"
)

# Create GluonTS datasets and split it
def split_data(df, prediction_length, windows=1):
    # Ensure the df index is datetime type
    df.index = pd.to_datetime(df.index)

    # Create GluonTS format dataset
    start_timestamp = pd.Period(df.index[0], "D")
    dataset = [
        {
            "start": start_timestamp,
            "target": df["close"].values.astype(np.float32),
        }
    ]

    # Split the data (!note the negative symbol in front of prediction length!)
    train_data, test_gen = split(dataset, offset=-prediction_length)

    # Generate test instances
    test_data = test_gen.generate_instances(prediction_length, windows=windows)

    return dataset, train_data, test_data


prediction_length = 64

# Create the estimator

estimator = DeepAREstimator(freq="D",
                            # context length is number of time steps will look back(5 days in a week)
                            context_length=512,
                            prediction_length=prediction_length,
                            num_layers=2,
                            lr= 0.001,  # Sama dengan learning rate pada Chronos
                            num_parallel_samples= 20,  # Sama dengan jumlah sampel Chronos
                            trainer_kwargs={"max_epochs": 50,"ctx": gpu(0)}
                            )

# Train on training dataset
dataset, training_data, test_data = split_data(df, prediction_length)

# train model dataset
train_model_data = ListDataset(
    dataset_train_model,
    freq = "D"
)

model = estimator.train(train_model_data, num_workers=0)


# Make forecast
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset, predictor=model
)

forecasts = list(forecast_it)
tss = list(ts_it)

# Calculate accuracy metrics
evaluator = Evaluator()
evaluator(tss, forecasts)
agg_metrics, item_metrics = evaluator(
    iter(tss), iter(forecasts), num_series=len(dataset)
)

/home/yogi/miniconda3/envs/baseline/lib/python3.12/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/tmp/ipykernel_1474780/1014250511.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method=fillna_method, inplace=True)
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/yogi/miniconda3/envs/baseline/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connec

Epoch 0: |          | 0/? [00:00<?, ?it/s] 

OutOfMemoryError: CUDA out of memory. Tried to allocate 66.00 MiB. GPU 0 has a total capacity of 11.76 GiB of which 60.31 MiB is free. Process 1230636 has 7.87 GiB memory in use. Process 1435290 has 3.67 GiB memory in use. Including non-PyTorch memory, this process has 154.00 MiB memory in use. Of the allocated memory 18.48 MiB is allocated by PyTorch, and 5.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
item_metrics

In [ ]:
from gluonts.dataset.split import split


from gluonts.ev.metrics import MASE, MeanWeightedSumQuantileLoss
from gluonts.model.evaluation import evaluate_forecasts

metrics_df = evaluate_forecasts(
    forecasts,
    test_data=test_data,
    metrics=[
        MASE(),
        MeanWeightedSumQuantileLoss(np.arange(0.1, 1.0, 0.1)),
    ],
)

metrics_df